In [111]:
import json
import numpy as np
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader 
#from torchvision import datasets, models, transforms
from PIL import Image
import torch
import torch.nn as nn
from timesformer_pytorch import TimeSformer  
# Load data

with open("/home/yetian/glaucoma_progression/uwhvf/alldata.json") as fin:
  dat = json.loads(fin.read())

# Basic statistics

print(f"Total of {dat['pts']} patients, {dat['eyes']} eyes, and {dat['hvfs']} HVFs")
# Expected output: Total of 3871 patients, 7428 eyes, and 28943 HVFs

Total of 3871 patients, 7428 eyes, and 28943 HVFs


In [112]:
datalist = [] #7248, 3, 9, 9
labellist = [] #7428, 1

In [113]:
key_list = []
for key in dat['data'].keys():
    key_list.append(key)

In [114]:
def hundred_to_zero(temp2):
    for i in range(temp2.shape[0]):
        for j in range(temp2.shape[1]):
            if temp2[i][j] == 100:
                temp2[i][j] = float(0)
    return temp2

In [115]:
def duplicate(temp):
    a = temp[np.newaxis, :]
    a = np.repeat(a, 3, axis=0)
    return a

In [116]:
datalist = [] #7248, 3, 9, 9
labellist = [] #7428, 1
for key in key_list:
    if 'L' in dat['data'][key].keys():  #['L'] or ['R']
        if len(dat['data'][key]['L']) > 3:  # at least 4 frames
            temp0 = np.array(dat['data'][key]['L'][0]['td'])
            temp1 = np.array(dat['data'][key]['L'][1]['td'])
            temp2 = np.array(dat['data'][key]['L'][2]['td'])

            newtemp0 = np.pad(temp0, pad_width=((0,1),(0,0)), mode='constant')   #9,9
            newtemp0 = hundred_to_zero(newtemp0)
            newtemp0 = duplicate(newtemp0)                                       # 3,9,9
            newtemp0 = newtemp0[np.newaxis,:]

            newtemp1 = np.pad(temp1, pad_width=((0,1),(0,0)), mode='constant')
            newtemp1 = hundred_to_zero(newtemp1)
            newtemp1 = duplicate(newtemp1)
            newtemp1 = newtemp1[np.newaxis,:]

            newtemp2 = np.pad(temp2, pad_width=((0,1),(0,0)), mode='constant')
            newtemp2 = hundred_to_zero(newtemp2)
            newtemp2 = duplicate(newtemp2)
            newtemp2 = newtemp2[np.newaxis,:]

            video = np.vstack((newtemp0, newtemp1, newtemp2))                    # 3, 3, 9, 9  #frame, channel, H, W

            datalist.append(video)

            total_num_of_frames = len(dat['data'][key]['L'])
            MD_diff_list = []
            for i in range(0, total_num_of_frames - 1):
                age1 = dat['data'][key]['L'][i]['age']
                age2 = dat['data'][key]['L'][i+1]['age']
                temp1 = np.array(dat['data'][key]['L'][i]['td'])
                temp1 = hundred_to_zero(temp1)
                
                temp2 = np.array(dat['data'][key]['L'][i+1]['td'])
                temp2 = hundred_to_zero(temp2)
                
                MD1 = sum(sum(temp1))/np.count_nonzero(temp1)
                MD2 = sum(sum(temp2))/np.count_nonzero(temp2)
                MD_diff_per_year = (MD2-MD1)/(age2-age1)
              #  db = 10 * np.log10(abs(MD1 - MD2))
              #  db_per_year = db/abs(age1-age2)
                MD_diff_list.append(MD_diff_per_year)
            labellist.append(MD_diff_list)


for key in key_list:
    if 'R' in dat['data'][key].keys():  #['L'] or ['R']
        if len(dat['data'][key]['R']) > 3:  # at least 4 frames
            temp0 = np.array(dat['data'][key]['R'][0]['td'])
            temp1 = np.array(dat['data'][key]['R'][1]['td'])
            temp2 = np.array(dat['data'][key]['R'][2]['td'])

            newtemp0 = np.pad(temp0, pad_width=((0,1),(0,0)), mode='constant')   #9,9
            newtemp0 = hundred_to_zero(newtemp0)
            newtemp0 = duplicate(newtemp0)                                       # 3,9,9
            newtemp0 = newtemp0[np.newaxis,:]

            newtemp1 = np.pad(temp1, pad_width=((0,1),(0,0)), mode='constant')
            newtemp1 = hundred_to_zero(newtemp1)
            newtemp1 = duplicate(newtemp1)
            newtemp1 = newtemp1[np.newaxis,:]

            newtemp2 = np.pad(temp2, pad_width=((0,1),(0,0)), mode='constant')
            newtemp2 = hundred_to_zero(newtemp2)
            newtemp2 = duplicate(newtemp2)
            newtemp2 = newtemp2[np.newaxis,:]

            video = np.vstack((newtemp0, newtemp1, newtemp2))                    # 3, 3, 9, 9  #frame, channel, H, W

            datalist.append(video)
            
            total_num_of_frames = len(dat['data'][key]['R'])
            MD_diff_list = []
            for i in range(0, total_num_of_frames - 1):
                age1 = dat['data'][key]['R'][i]['age']
                age2 = dat['data'][key]['R'][i+1]['age']
                temp1 = np.array(dat['data'][key]['R'][i]['td'])
                temp1 = hundred_to_zero(temp1)
                temp2 = np.array(dat['data'][key]['R'][i+1]['td'])
                temp2 = hundred_to_zero(temp2)
                MD1 = sum(sum(temp1))/np.count_nonzero(temp1)
                MD2 = sum(sum(temp2))/np.count_nonzero(temp2)
                MD_diff_per_year = (MD2-MD1)/(age2-age1)
               # db = 10 * np.log10(abs(MD1 - MD2))
               # db_per_year = db/abs(age1-age2)
                MD_diff_list.append(MD_diff_per_year)
            labellist.append(MD_diff_list)


In [117]:
print(len(datalist)) # datalist shape = (3,3,9,9)
print(len(labellist))

2991
2991


total 7428 eyes


over 4 frames: 2065

over 3 frames: 2991

over 2 frames: 4452

In [118]:
count = 0
for MD_diff in labellist:
    if min(MD_diff) < -1:  # 10log(-1.5) : -0.84  <---> wrong?
        count +=1

print('total',len(labellist))
print('progressing', count)
print('ratio of progressing:','{:.3f}'.format(count/len(labellist)))

total 2991
progressing 2200
ratio of progressing: 0.736


In [119]:
labellist[0]  #'647', 11 frames

[2.007726415094342,
 -1.7920555879746738,
 -0.5054338060393485,
 0.30318995208146227,
 -0.29911762885747256,
 -0.98614218328841,
 1.5352219820056094,
 -1.1572355217284445,
 -0.6236105572845692,
 0.7072336212112875]

Another example of MD changes:

In [120]:
labellist[2]

[0.7955493752725599,
 -0.17475278744884137,
 -0.07990652966228588,
 -0.3885928926099877,
 0.7233696285832772,
 -1.765963380435085,
 1.2162008294126148,
 0.5923301801744322,
 0.21244461046655855]

In [121]:
progression_labellist = []

for i in range(len(labellist)):
    if min(labellist[i]) < -0.84:
        progression_labellist.append(int(1))
    else:
        progression_labellist.append(int(0))

#print(progression_labellist)
print('total number of over 3 frame subjects:', len(progression_labellist))
print('among them, progressing number is:', progression_labellist.count(1))

total number of over 3 frame subjects: 2991
among them, progressing number is: 2353


TimeSformer

In [122]:
# defining the Dataset class
class VFDataset(Dataset):
    def __init__(self, label, img, transform=None):
        self.label = label
        self.img = img
        self.transform = transform

    def __len__(self):
        return len(self.label)
  
    def __getitem__(self, index):
        image = self.img[index]
        label = self.label[index]

        return image, label


In [123]:
VF_datasets = {
    'train': 
    VFDataset(img = datalist, label = progression_labellist,
                transform=None),
}

train_set, val_set = torch.utils.data.random_split(VF_datasets['train'], [2393, 598])

dataloaders = {
    'train':
    torch.utils.data.DataLoader(train_set,
                                batch_size=5,
                                shuffle=True),
    'validation':
    torch.utils.data.DataLoader(val_set,
                                #val_set,
                                
                                batch_size=5,
                                shuffle=False)
}



In [124]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device

In [125]:


class TMSFM(nn.Module):
  def __init__(self):
    super(TMSFM, self).__init__()
    self.model = TimeSformer(
        dim = 512,
        image_size = 9,
        patch_size = 3,
        num_frames = 3,
        num_classes = 1,
        depth = 12,
        heads = 8,
        dim_head =  64,
        attn_dropout = 0,
        ff_dropout = 0
    )
  #  self.model.Linear = nn.Linear(512, out_features = 1)
 #   self.fc = nn.Linear(2,1)

  def forward(self, x):
    #self.model = nn.ModuleList()
    x = self.model(x)
    
 #   x = self.fc(x)

    return x

In [126]:
model = TMSFM() 
#model.to(device)
print(model)

TMSFM(
  (model): TimeSformer(
    (to_patch_embedding): Linear(in_features=27, out_features=512, bias=True)
    (frame_rot_emb): RotaryEmbedding()
    (image_rot_emb): AxialRotaryEmbedding()
    (layers): ModuleList(
      (0): ModuleList(
        (0): PreNorm(
          (fn): Attention(
            (to_qkv): Linear(in_features=512, out_features=1536, bias=False)
            (to_out): Sequential(
              (0): Linear(in_features=512, out_features=512, bias=True)
              (1): Dropout(p=0, inplace=False)
            )
          )
          (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (1): PreNorm(
          (fn): Attention(
            (to_qkv): Linear(in_features=512, out_features=1536, bias=False)
            (to_out): Sequential(
              (0): Linear(in_features=512, out_features=512, bias=True)
              (1): Dropout(p=0, inplace=False)
            )
          )
          (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=T

Multi-frame prediction example:

In [127]:
video = torch.randn(5, 3, 3, 9,9)
pred = model(video)
pred

tensor([[1.1464],
        [0.7509],
        [0.8525],
        [0.8431],
        [0.7729]], grad_fn=<AddmmBackward0>)

In [128]:
from torch.optim.lr_scheduler import StepLR
import torch.optim as optim
from tqdm import tqdm

criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(0.7))
optimizer = optim.Adam(model.parameters(), lr=1e-5)

In [129]:
import time
import copy

def train_model(model, criterion, optimizer, num_epochs=3):
    since = time.time()

    best_f1 = 0

    train_f1 = []
    val_f1 = []
    train_accuracy = []
    val_accuracy = []

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            train_outputs = []
            train_preds = []
            train_trues = []
            output_tensors = []



            for inputs, labels in tqdm(dataloaders[phase]):

                labels = labels.unsqueeze(1)

                # inputs = inputs.to(device)
                # labels = labels.to(device)

               # print(inputs.shape)
               # print(labels.shape)

                outputs = model(inputs.float())
              #  print(outputs.shape)
              #  print('outputs:', outputs)
              
                loss = criterion(outputs, labels.float())
              #  print('loss')

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

               # _, preds = torch.max(outputs, 1)
                preds = (outputs >= 0)   #not 0.5!!!
                #preds_array = np.asarray(preds)
                preds_array = preds.cpu().numpy()                #TypeError: can’t convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first
                #labels_array = np.asarray(labels.data)
                labels_array = labels.data.cpu().numpy()
                outputs_array = outputs.detach().cpu().numpy()

                train_outputs.extend(outputs_array) #in tensor
                train_preds.extend(preds_array)
                train_trues.extend(labels_array)
                output_tensors.extend(outputs.detach().cpu())

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                


                # from torchmetrics.functional import precision_recall
                # precision, recall = precision_recall(preds, labels.data, average='macro', num_classes=2)
            #    print('pred',preds)
            #    print('dt',labels.data)

            epoch_loss = running_loss / len(train_set)
            epoch_acc = running_corrects.double() / len(val_set)

            

            print('pred array 10:', train_preds[0:10])
            print('output array 10:', output_tensors[0:10])
            print('trues array 10:', train_trues[0:10])

            from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, roc_curve, auc, roc_auc_score
            sklearn_accuracy = accuracy_score(train_trues, train_preds) 
            sklearn_precision = precision_score(train_trues, train_preds, average='macro')
            sklearn_recall = recall_score(train_trues, train_preds, average='macro')
            sklearn_f1 = f1_score(train_trues, train_preds, average='macro')
            
            output_tensors = torch.sigmoid(torch.tensor(output_tensors)).numpy()
            fpr, tpr, thresholds = roc_curve(train_trues, output_tensors, pos_label=1)
            previous_auc = auc(fpr, tpr)
            sklearn_auc = roc_auc_score(train_trues, output_tensors)

            #print(train_trues[0:10], train_preds[0:10])

            print('{} loss: {:.4f}, torch acc: {:.4f}, sklearn acc: {:.4f}, precision:{:.4f}, recall: {:.4f}, f1_score:{:.4f}, auc:{:.4f}, previous auc:{:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc,
                                                        sklearn_accuracy,
                                                        sklearn_precision,
                                                        sklearn_recall,
                                                        sklearn_f1,
                                                        sklearn_auc,
                                                        previous_auc))
            if phase == 'train':
              train_f1.append(sklearn_f1)
              train_accuracy.append(sklearn_accuracy)


            if phase == 'validation':
              target_names = ['healthy', 'disease']
              print(classification_report(train_trues, train_preds, target_names=target_names))

              val_f1.append(sklearn_f1)
              val_accuracy.append(sklearn_accuracy)

            if phase == 'validation' and sklearn_f1 > best_f1:
      #          torch.save(model, '/content/drive/MyDrive/research/best_model/best-model-resnet101.pt')
      #          torch.save(model.state_dict(), '/content/drive/MyDrive/research/best_model/best-model-parameters-resnet101.pt')
                # method I: plt
                save_outputs = output_tensors
                save_labels = train_trues
                save_preds = train_preds


                import matplotlib.pyplot as plt
                plt.title('Receiver Operating Characteristic')
                plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % previous_auc)
                plt.legend(loc = 'lower right')
                plt.plot([0, 1], [0, 1],'r--')
                plt.xlim([0, 1])
                plt.ylim([0, 1])
                plt.ylabel('True Positive Rate')
                plt.xlabel('False Positive Rate')
                plt.show()


           
                best_f1 = sklearn_f1
                best_model_wts = copy.deepcopy(model.state_dict())
                print('best_f1:', best_f1)
                print('A new best model saved at epoch {}!'.format(epoch + 1))


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val F1: {:4f}'.format(best_f1))            

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, train_f1, train_accuracy, val_f1, val_accuracy, save_outputs, save_labels, save_preds

In [ ]:
model_trained = train_model(model, criterion, optimizer, num_epochs=50)